In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../"))

# Any results you write to the current directory are saved as output.

['.idea', '.ipynb_checkpoints', '.vscode', 'code before 2019-03-21', 'compare', 'DataMing', 'HuaWei', 'JData', 'JData_Predict', 'Maching Learning', 'Markdown', 'out PDF', 'tiancheng', 'venv', '__pycache__']


In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
path = './'

In [ ]:
import lightgbm as lgb
pd.set_option('display.max_columns',None)
user_app_actived_reader = pd.read_csv(path + "user_app_actived.csv", names=['uid','appId'], iterator=True)

In [4]:
app_id_used = []
app_id_frequence = dict()
i = 1
while True:
    try:
        print("index: {}".format(i))
        i+=1
        df = user_app_actived_reader.get_chunk(100000)
        for index, row in df.iterrows():
            temp = row['appId'].split('#')
            app_id_used += temp
#             app_id_used = list(set(app_id_used))
#             for app_id_key in temp:
#                 app_id_frequence[app_id_key] = 1
#                 app_id_frequence[app_id_key] = 1 if app_id_key not in app_id_frequence.keys() else app_id_frequence[app_id_key] + 1
    except StopIteration:
        break

index: 1
index: 2
index: 3
index: 4
index: 5
index: 6
index: 7
index: 8
index: 9
index: 10
index: 11
index: 12
index: 13
index: 14
index: 15
index: 16
index: 17
index: 18
index: 19
index: 20
index: 21
index: 22
index: 23
index: 24
index: 25
index: 26
index: 27


In [5]:
user_app_actived_reader = []

In [6]:
from collections import Counter
counter = Counter(app_id_used)

In [7]:
counter = pd.DataFrame(data={
    'appId': list(counter.keys()),
    'app_fre': list(counter.values())
})

In [8]:
counter.describe()

,app_fre
count,9.401000e+03
mean,1.082897e+04
std,8.089038e+04
min,1.000000e+00
25%,4.790000e+02
50%,1.063000e+03
75%,2.853000e+03
max,2.339048e+06


In [9]:
id2fre_dict = dict(zip(counter.appId.values, counter.app_fre.values))

In [10]:
app_id_used = set(counter.appId.values)

In [11]:
app_info = pd.read_csv(path + 'app_info.csv', names=['appId','category'])

In [12]:
app_id_exist = set(app_info.appId.values)

In [13]:
app_id_not_exist = app_id_used.difference(app_id_exist)
app_id_intersection = app_id_exist.intersection(app_id_used)

In [14]:
print(len(app_id_not_exist), len(app_id_intersection), len(app_id_exist), len(app_id_used))

1232 8169 167622 9401


In [15]:
df_intersection = pd.DataFrame()
df_intersection['appId'] = list(app_id_intersection)
df_intersection.head()

,appId
0,a00129139
1,a00176868
2,a00444387
3,a0044927
4,a00108271


In [16]:
df_intersection = pd.merge(df_intersection, app_info, on='appId', how='left')

In [17]:
df_intersection.describe()

,appId,category
count,10618,10618
unique,8169,31
top,a00313693,实用工具
freq,8,2782


In [18]:
df_diff = pd.DataFrame()

**暂时作为未知进行处理，后面考虑进行分类别？**

In [19]:
df_diff['appId'] = list(app_id_not_exist)
# 暂时作为未知进行处理
df_diff['category'] = '未知'
app_info_used = df_intersection.append(df_diff)

In [20]:
app_info_used.describe()

,appId,category
count,11850,11850
unique,9401,32
top,a00313693,实用工具
freq,8,2782


In [21]:
app_info_used[app_info_used.appId == 'a00313693']

,appId,category
6123,a00313693,便捷生活
6124,a00313693,出行导航
6125,a00313693,动作射击
6126,a00313693,实用工具
6127,a00313693,教育
6128,a00313693,购物比价
6129,a00313693,运动健康
6130,a00313693,金融理财


In [22]:
app_info_used['frequency'] = app_info_used['appId'].apply(lambda x: id2fre_dict[x])

In [23]:
app_info_appId_counter = Counter(app_info_used['appId'].values)

In [24]:
app_info_used['category_num'] = app_info_used['appId'].apply(lambda x: app_info_appId_counter[x])

In [25]:
app_info_used.head()

,appId,category,frequency,category_num
0,a00129139,实用工具,357,1
1,a00176868,便捷生活,808,1
2,a00444387,实用工具,1179,1
3,a0044927,角色扮演,323,1
4,a00108271,实用工具,755,2


添加category_id字段，也就是category映射到32个字段上，同时添加frequency_inv字段，**暂时取2512500/frequency** 2512500是总人数

In [26]:
category2id = dict(zip(np.unique(app_info_used.category.values), np.arange(1, 33)))

In [27]:
category2id

{'主题个性': 1,
 '休闲游戏': 2,
 '休闲益智': 3,
 '体育竞速': 4,
 '便捷生活': 5,
 '儿童': 6,
 '出行导航': 7,
 '动作冒险': 8,
 '动作射击': 9,
 '商务': 10,
 '图书阅读': 11,
 '学习办公': 12,
 '实用工具': 13,
 '影音娱乐': 14,
 '拍摄美化': 15,
 '教育': 16,
 '新闻阅读': 17,
 '旅游住宿': 18,
 '未知': 19,
 '棋牌天地': 20,
 '棋牌桌游': 21,
 '汽车': 22,
 '益智棋牌': 23,
 '社交通讯': 24,
 '经营策略': 25,
 '网络游戏': 26,
 '美食': 27,
 '表盘个性': 28,
 '角色扮演': 29,
 '购物比价': 30,
 '运动健康': 31,
 '金融理财': 32}

In [28]:
app_info_used['category_id'] = app_info_used.category.apply(lambda x: category2id[x])
app_info_used['frequency_inv'] = 2512500/app_info_used['frequency']

In [29]:
app_info_used.head()

,appId,category,frequency,category_num,category_id,frequency_inv
0,a00129139,实用工具,357,1,13,7037.815126
1,a00176868,便捷生活,808,1,5,3109.529703
2,a00444387,实用工具,1179,1,13,2131.043257
3,a0044927,角色扮演,323,1,29,7778.637771
4,a00108271,实用工具,755,2,13,3327.814570


In [30]:
app_info_used = app_info_used.sort_values(by=['appId'])

In [31]:
app_info_used.head()

,appId,category,frequency,category_num,category_id,frequency_inv
599,\N,未知,3738,1,19,672.150883
10459,a00100007,商务,3214,2,10,781.736154
10458,a00100007,便捷生活,3214,2,5,781.736154
388,a00100109,未知,572,1,19,4392.482517
1910,a00100361,教育,701,1,16,3584.165478


In [32]:
app_info_used.index = np.arange(0, len(app_info_used.index))

In [33]:
app_info_used.head()

,appId,category,frequency,category_num,category_id,frequency_inv
0,\N,未知,3738,1,19,672.150883
1,a00100007,商务,3214,2,10,781.736154
2,a00100007,便捷生活,3214,2,5,781.736154
3,a00100109,未知,572,1,19,4392.482517
4,a00100361,教育,701,1,16,3584.165478


In [34]:
id2category_dict = dict()
id2freq = dict()
id2category_num = dict()
for index, row in app_info_used.iterrows():
    if row['appId'] in id2category_dict:
        id2category_dict[row['appId']] += [row['category_id']]
        
    else:
        id2category_dict[row['appId']] = [row['category_id']]
        id2freq[row['appId']] = row['frequency_inv']
        id2category_num[row['appId']] = row['category_num']

## id2category_dict = dict()  app_id -> category
## id2freq = dict()           app_id -> freq
## id2category_num = dict()   app_id -> category_num

## app_info_used 是app信息表格

In [35]:
age_test = pd.read_csv(path + 'age_test.csv', names = ['uId'])
age_train = pd.read_csv(path + 'age_train.csv', names = ['uId'], usecols=[0])
age_test = list(age_test.uId.values)
age_uid_total = age_test + list(age_train.uId.values)
age_uid_total = set(age_uid_total)
app_actived = pd.read_csv(path + 'user_app_actived.csv', header=None, names = ['uId', 'appId'])
app_actived_uid = set(app_actived.uId.values)

In [36]:
app_actived

,uId,appId
0,1000008,a00105157#a00109386#a00134746#a0013952#a001451...
1,1000010,a00102782#a00107077#a00109386#a00134746#a00136...
2,1000011,a00158535#a00163116#a00170432#a00187480#a00224...
3,1000013,a00109386#a0011601#a0012768#a00136287#a0014516...
4,1000014,a00109386#a0011894#a0013059#a00134840#a0013578...
5,1000016,a00102782#a0011894#a002265#a00239261#a00289791...
6,1000021,a00109386#a00145168#a0015359#a0021880#a0023501...
7,1000022,a00109386#a00117437#a00118538#a0013770#a001451...
8,1000036,a00101827#a00101845#a00107264#a00109386#a00116...
9,1000047,a00109386#a00143562#a00200080#a0021880#a002244...


没有缺失值，这是一件值得开心的事情

In [37]:
len(age_uid_total.difference(app_actived_uid))

0

In [38]:
app_actived_uid == age_uid_total

True

In [39]:
len(app_actived_uid)

2512500

用户表 添加 category_num   [1, 2, 3, 4, 5, 6, 8] 7个 字段、添加category  1-32个字段

In [40]:
category_index = ['category' + str(i + 1) for i in range(32)]
len(category_index)
category_num_index = ['category_num' + str(i) for i in [1, 2, 3, 4, 5, 6, 8]]
len(category_num_index)
new_index = category_index + category_num_index


In [41]:
for index in new_index:
    app_actived[index] = 0
    app_actived[index] = app_actived[index].astype(int)

In [42]:
app_actived['fre'] = 0.0

In [43]:
app_actived.head()

,uId,appId,category1,category2,category3,category4,category5,category6,category7,category8,category9,category10,category11,category12,category13,category14,category15,category16,category17,category18,category19,category20,category21,category22,category23,category24,category25,category26,category27,category28,category29,category30,category31,category32,category_num1,category_num2,category_num3,category_num4,category_num5,category_num6,category_num8,fre
0,1000008,a00105157#a00109386#a00134746#a0013952#a001451...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,1000010,a00102782#a00107077#a00109386#a00134746#a00136...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2,1000011,a00158535#a00163116#a00170432#a00187480#a00224...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3,1000013,a00109386#a0011601#a0012768#a00136287#a0014516...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,1000014,a00109386#a0011894#a0013059#a00134840#a0013578...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [44]:
max(id2category_num.values())

8

In [45]:
# id2category_dict = dict()
# id2freq = dict()
# id2category_num = dict()
def get_statis(app_id_list):
    app_id = app_id_list.split('#')
    temps = np.zeros(39, dtype=int)
    freq = 0
    for i in app_id:
        temps[np.array(id2category_dict[i]) - 1] += 1
        freq += id2freq[i]
        temp_num = id2category_num[i]
        if temp_num == 8:
            temp_num = 7
        temps[temp_num + 32 - 1] += 1
#     temps = np.array(list(temps).append(freq))
    return list(temps) + [freq]

In [46]:
index = new_index + ['fre']

In [47]:
temp = app_actived.appId.apply(lambda x: get_statis(x))

In [48]:
len(temp[1])

40

In [49]:
len(index)

40

In [50]:
temp2 = app_actived[index].iloc[0]

In [51]:
temp2[index] = temp[1]

In [58]:
app2fea_dict = dict(zip(app_actived.uId.values, temp))

In [59]:
app_actived[index] = np.vstack(app_actived.uId.apply(lambda x:np.array(app2fea_dict[x])).values)
app_actived.head()

,uId,appId,category1,category2,category3,category4,category5,category6,category7,category8,category9,category10,category11,category12,category13,category14,category15,category16,category17,category18,category19,category20,category21,category22,category23,category24,category25,category26,category27,category28,category29,category30,category31,category32,category_num1,category_num2,category_num3,category_num4,category_num5,category_num6,category_num8,fre
0,1000008,a00105157#a00109386#a00134746#a0013952#a001451...,0.0,0.0,0.0,0.0,12.0,2.0,6.0,0.0,0.0,5.0,0.0,0.0,13.0,5.0,0.0,5.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,7.0,0.0,0.0,1.0,0.0,0.0,8.0,1.0,6.0,24.0,17.0,2.0,3.0,0.0,0.0,0.0,4057.887938
1,1000010,a00102782#a00107077#a00109386#a00134746#a00136...,0.0,0.0,0.0,0.0,15.0,0.0,11.0,0.0,0.0,1.0,0.0,0.0,16.0,6.0,3.0,4.0,4.0,4.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,3.0,20.0,17.0,5.0,3.0,0.0,0.0,0.0,3841.639311
2,1000011,a00158535#a00163116#a00170432#a00187480#a00224...,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,5.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,8.0,6.0,0.0,2.0,0.0,0.0,0.0,2741.710736
3,1000013,a00109386#a0011601#a0012768#a00136287#a0014516...,0.0,0.0,0.0,0.0,11.0,0.0,7.0,0.0,0.0,2.0,0.0,0.0,13.0,5.0,0.0,3.0,3.0,3.0,2.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,5.0,24.0,14.0,4.0,1.0,0.0,0.0,0.0,5540.761669
4,1000014,a00109386#a0011894#a0013059#a00134840#a0013578...,0.0,0.0,0.0,0.0,15.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,25.0,10.0,0.0,10.0,5.0,2.0,1.0,0.0,0.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,4.0,30.0,18.0,4.0,3.0,0.0,0.0,0.0,4246.621297


In [ ]:
%whos

In [60]:
app_actived['app_num'] = app_actived.appId.apply(lambda x: len(x.split('#')))

In [61]:
app_actived.head()

,uId,appId,category1,category2,category3,category4,category5,category6,category7,category8,category9,category10,category11,category12,category13,category14,category15,category16,category17,category18,category19,category20,category21,category22,category23,category24,category25,category26,category27,category28,category29,category30,category31,category32,category_num1,category_num2,category_num3,category_num4,category_num5,category_num6,category_num8,fre,app_num
0,1000008,a00105157#a00109386#a00134746#a0013952#a001451...,0.0,0.0,0.0,0.0,12.0,2.0,6.0,0.0,0.0,5.0,0.0,0.0,13.0,5.0,0.0,5.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,7.0,0.0,0.0,1.0,0.0,0.0,8.0,1.0,6.0,24.0,17.0,2.0,3.0,0.0,0.0,0.0,4057.887938,46
1,1000010,a00102782#a00107077#a00109386#a00134746#a00136...,0.0,0.0,0.0,0.0,15.0,0.0,11.0,0.0,0.0,1.0,0.0,0.0,16.0,6.0,3.0,4.0,4.0,4.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,3.0,20.0,17.0,5.0,3.0,0.0,0.0,0.0,3841.639311,45
2,1000011,a00158535#a00163116#a00170432#a00187480#a00224...,0.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0,5.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,8.0,6.0,0.0,2.0,0.0,0.0,0.0,2741.710736,16
3,1000013,a00109386#a0011601#a0012768#a00136287#a0014516...,0.0,0.0,0.0,0.0,11.0,0.0,7.0,0.0,0.0,2.0,0.0,0.0,13.0,5.0,0.0,3.0,3.0,3.0,2.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,7.0,2.0,5.0,24.0,14.0,4.0,1.0,0.0,0.0,0.0,5540.761669,43
4,1000014,a00109386#a0011894#a0013059#a00134840#a0013578...,0.0,0.0,0.0,0.0,15.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,25.0,10.0,0.0,10.0,5.0,2.0,1.0,0.0,0.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,4.0,30.0,18.0,4.0,3.0,0.0,0.0,0.0,4246.621297,55


## 转变存储格式

In [62]:
app_actived['app_num'] = app_actived['app_num'].astype(int)
app_actived[new_index] = app_actived[new_index].astype(int)

# 丢弃app_id列
app_actived.drop(columns = 'appId', inplace=True)
app_actived.head()

,uId,category1,category2,category3,category4,category5,category6,category7,category8,category9,category10,category11,category12,category13,category14,category15,category16,category17,category18,category19,category20,category21,category22,category23,category24,category25,category26,category27,category28,category29,category30,category31,category32,category_num1,category_num2,category_num3,category_num4,category_num5,category_num6,category_num8,fre,app_num
0,1000008,0,0,0,0,12,2,6,0,0,5,0,0,13,5,0,5,1,1,1,0,0,2,0,7,0,0,1,0,0,8,1,6,24,17,2,3,0,0,0,4057.887938,46
1,1000010,0,0,0,0,15,0,11,0,0,1,0,0,16,6,3,4,4,4,1,0,0,0,0,4,0,0,0,0,0,7,2,3,20,17,5,3,0,0,0,3841.639311,45
2,1000011,0,0,1,0,4,0,0,0,1,0,0,0,6,5,0,1,2,0,0,0,0,0,0,3,1,0,0,0,1,3,0,0,8,6,0,2,0,0,0,2741.710736,16
3,1000013,0,0,0,0,11,0,7,0,0,2,0,0,13,5,0,3,3,3,2,0,0,0,0,5,0,0,0,0,0,7,2,5,24,14,4,1,0,0,0,5540.761669,43
4,1000014,0,0,0,0,15,0,4,0,0,1,0,0,25,10,0,10,5,2,1,0,0,1,0,6,0,0,0,0,0,6,0,4,30,18,4,3,0,0,0,4246.621297,55


In [63]:
app_actived.to_csv('./output/app_actived_new.csv', index=False)

In [64]:
app_actived.describe()

,uId,category1,category2,category3,category4,category5,category6,category7,category8,category9,category10,category11,category12,category13,category14,category15,category16,category17,category18,category19,category20,category21,category22,category23,category24,category25,category26,category27,category28,category29,category30,category31,category32,category_num1,category_num2,category_num3,category_num4,category_num5,category_num6,category_num8,fre,app_num
count,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06,2.512500e+06
mean,2.256250e+06,1.755471e-01,4.035821e-04,5.382038e-01,4.770149e-02,9.789607e+00,2.506846e-01,2.960330e+00,1.910448e-05,3.170523e-01,1.978901e+00,4.087562e-04,7.644975e-03,1.634636e+01,7.468680e+00,1.743654e+00,4.224839e+00,2.782909e+00,7.657421e-01,1.710926e+00,1.198647e-02,2.160756e-01,4.513409e-01,3.992040e-04,4.733523e+00,3.594113e-01,1.209711e-02,8.024808e-01,2.189055e-05,8.456517e-02,4.669950e+00,5.061807e-01,4.039176e+00,2.150519e+01,1.391111e+01,2.984884e+00,2.007852e+00,3.952239e-04,9.630408e-02,1.294289e-02,9.401000e+03,4.051868e+01
std,7.252964e+05,4.574309e-01,2.008530e-02,1.054482e+00,2.457641e-01,6.092056e+00,1.670422e+00,2.994119e+00,4.370826e-03,7.058737e-01,1.906636e+00,2.021359e-02,8.710070e-02,8.488256e+00,4.214684e+00,2.001112e+00,2.852492e+00,2.354522e+00,1.256042e+00,1.580852e+00,1.088246e-01,5.423645e-01,8.751460e-01,2.132525e-02,2.912475e+00,6.717153e-01,1.098897e-01,1.362827e+00,4.678683e-03,3.821107e-01,3.164112e+00,9.489601e-01,4.592366e+00,1.314513e+01,8.583999e+00,1.977834e+00,7.203497e-01,1.987631e-02,2.989307e-01,1.130282e-01,1.925124e+04,2.283286e+01
min,1.000001e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.074155e+00,1.000000e+00
25%,1.628126e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+01,4.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,1.000000e+00,1.200000e+01,8.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.976985e+03,2.400000e+01
50%,2.256250e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,1.500000e+01,7.000000e+00,1.000000e+00,4.000000e+00,2.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,0.000000e+00,3.000000e+00,1.900000e+01,1.300000e+01,3.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.596049e+03,3.700000e+01
75%,2.884375e+06,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.300000e+01,0.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,2.100000e+01,1.000000e+01,2.000000e+00,5.000000e+00,4.000000e+00,1

In [66]:
app_info_used.to_csv('./output/app_info_used.csv', index=False)

## app_actived处理完毕
接下来进行marge然后训练模型

In [65]:
%whos

Variable                  Type         Data/Info
------------------------------------------------
Counter                   type         <class 'collections.Counter'>
age_test                  list         n=502500
age_train                 DataFrame                 uId\n0      <...>2010000 rows x 1 columns]
age_uid_total             set          {1000001, 1000002, 100000<...>512498, 3512499, 3512500}
app2fea_dict              dict         n=2512500
app_actived               DataFrame                 uId  categor<...>512500 rows x 42 columns]
app_actived_uid           set          {1000001, 1000002, 100000<...>512498, 3512499, 3512500}
app_id_exist              set          {'a00208638', 'a0053918',<...> 'a0017432', 'a00100149'}
app_id_frequence          dict         n=0
app_id_intersection       set          {'a00129139', 'a00176868'<...>'a00272066', 'a00180107'}
app_id_not_exist          set          {'a0046340', 'a00309002',<...>'a00248339', 'a00244801'}
app_id_used               se

# 第二次特征工程  添加安装频率处理

In [1]:
import numpy as np
import pandas as pd
import warnings 

In [2]:
path = 'output/'
app_info = pd.read_csv(path + 'app_info_used.csv')

In [3]:
app_info.head()

,appId,category,frequency,category_num,category_id,frequency_inv
0,\N,未知,3738,1,19,672.150883
1,a00100007,商务,3214,2,10,781.736154
2,a00100007,便捷生活,3214,2,5,781.736154
3,a00100109,未知,572,1,19,4392.482517
4,a00100361,教育,701,1,16,3584.165478


In [4]:
def deal_category_id():
    category_id_length = []
    unique_id = []
    diff_unique_id = []
    total_num = np.zeros([32, 10], dtype=int)
    for cat_id in  np.arange(1,33):
        temp = app_info[app_info.category_id == cat_id]['frequency'].values
        temp = np.sort(temp)
        length = len(temp)
        temp_index = []
        temp_values = []
        category_id_length += [length]
        if length < 10:
            unique_id += [cat_id] 
        else:
            diff_unique_id += [cat_id]
        print('category', cat_id, 'length is', length)
        for i in range(10):
            total_num[cat_id-1, i] = temp[i * length // 10]
            temp_index += [i * length // 10]
            temp_values += [temp[i * length // 10]]
        print(temp_index)
        print(temp_values)
    print(category_id_length)
    print('unique_id',unique_id)
    print('diff_unique_id', diff_unique_id)
    print(total_num)
    return category_id_length, unique_id, diff_unique_id, total_num

In [5]:
category_id_length, unique_id, diff_unique_id, total_num = deal_category_id()

category 1 length is 91
[0, 9, 18, 27, 36, 45, 54, 63, 72, 81]
[15, 104, 263, 412, 621, 784, 1201, 1460, 2401, 4608]
category 2 length is 2
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[142, 142, 142, 142, 142, 872, 872, 872, 872, 872]
category 3 length is 502
[0, 50, 100, 150, 200, 251, 301, 351, 401, 451]
[5, 134, 198, 268, 348, 457, 653, 912, 1489, 3580]
category 4 length is 58
[0, 5, 11, 17, 23, 29, 34, 40, 46, 52]
[6, 101, 154, 205, 256, 339, 583, 709, 1294, 2494]
category 5 length is 864
[0, 86, 172, 259, 345, 432, 518, 604, 691, 777]
[4, 429, 720, 942, 1272, 1703, 2449, 3906, 6921, 23339]
category 6 length is 415
[0, 41, 83, 124, 166, 207, 249, 290, 332, 373]
[82, 153, 220, 295, 435, 596, 817, 1059, 1427, 2626]
category 7 length is 375
[0, 37, 75, 112, 150, 187, 225, 262, 300, 337]
[29, 597, 888, 1162, 1554, 2170, 3039, 4317, 7855, 22942]
category 8 length is 1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[48, 48, 48, 48, 48, 48, 48, 48, 48, 48]
category 9 length is 228
[0, 22, 45, 68, 91, 114, 136, 159, 1

In [6]:
diff_unique_id = ['category_id' + str(i) + '_level' for i in diff_unique_id]
for cat_id in ['category_id' + str(i) + '_level' for i in np.arange(1, 33)]:
    app_info[cat_id] = 0
    app_info[cat_id] = app_info[cat_id].astype(int)

In [7]:
app_info.head()

,appId,category,frequency,category_num,category_id,frequency_inv,category_id1_level,category_id2_level,category_id3_level,category_id4_level,...,category_id23_level,category_id24_level,category_id25_level,category_id26_level,category_id27_level,category_id28_level,category_id29_level,category_id30_level,category_id31_level,category_id32_level
0,\N,未知,3738,1,19,672.150883,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,a00100007,商务,3214,2,10,781.736154,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,a00100007,便捷生活,3214,2,5,781.736154,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,a00100109,未知,572,1,19,4392.482517,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,a00100361,教育,701,1,16,3584.165478,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def get_level(frequency, category_id):
    temp = np.zeros(32,dtype=int)
    tag = 1
    if category_id in unique_id:
        tag = frequency
    else:
        for i in np.arange(1, 10):
            if frequency > total_num[category_id-1, i]:
                tag += 1
            else: 
                break
    temp[category_id - 1] = tag
    return temp

In [9]:
app_info[['category_id' + str(i) + '_level' for i in np.arange(1, 33)]] = np.vstack(app_info.apply(lambda x: get_level(x['frequency'], x['category_id']), axis=1))

In [10]:
app_info.describe()

,frequency,category_num,category_id,frequency_inv,category_id1_level,category_id2_level,category_id3_level,category_id4_level,category_id5_level,category_id6_level,...,category_id23_level,category_id24_level,category_id25_level,category_id26_level,category_id27_level,category_id28_level,category_id29_level,category_id30_level,category_id31_level,category_id32_level
count,1.185000e+04,11850.000000,11850.000000,1.185000e+04,11850.000000,11850.000000,11850.000000,11850.000000,11850.000000,11850.000000,...,11850.000000,11850.000000,11850.00000,11850.000000,11850.000000,11850.000000,11850.000000,11850.000000,11850.000000,11850.000000
mean,1.420502e+04,1.467679,15.830633,9.967332e+03,0.041857,0.085570,0.232574,0.026498,0.400591,0.192068,...,0.084641,0.258565,0.09173,2.564895,0.036287,0.004641,0.068270,0.183882,0.127004,0.348186
std,1.029941e+05,0.678705,8.106968,7.208983e+04,0.539224,8.115865,1.254077,0.427520,1.625484,1.142584,...,7.132708,1.319554,0.79396,245.508488,0.500181,0.505247,0.686472,1.118904,0.932793,1.521761
min,1.000000e+00,1.000000,1.000000,1.074155e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.122500e+02,1.000000,13.000000,7.805825e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.152000e+03,1.000000,14.000000,2.180990e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.218750e+03,2.000000,19.000000,4.904835e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.339048e+06,8.000000,32.000000,2.512500e+06,10.000000,872.000000,10.000000,10.000000,10.000000,10.000000,...,725.000000,10.000000,10.00000,26612.000000,10.000000,55.000000,10.000000,10.000000,10.000000,10.000000


In [11]:
cats = []
for i in range(1,33):
    cat = 'cat_num_' + str(i)
    cats += [cat]
    temp = dict(zip(app_info.appId[app_info.category_id == i].values, app_info['category_id' + str(i) + '_level'][app_info.category_id == i].values))
    exec(cat + '= %r' % temp)

In [12]:
cats

['cat_num_1',
 'cat_num_2',
 'cat_num_3',
 'cat_num_4',
 'cat_num_5',
 'cat_num_6',
 'cat_num_7',
 'cat_num_8',
 'cat_num_9',
 'cat_num_10',
 'cat_num_11',
 'cat_num_12',
 'cat_num_13',
 'cat_num_14',
 'cat_num_15',
 'cat_num_16',
 'cat_num_17',
 'cat_num_18',
 'cat_num_19',
 'cat_num_20',
 'cat_num_21',
 'cat_num_22',
 'cat_num_23',
 'cat_num_24',
 'cat_num_25',
 'cat_num_26',
 'cat_num_27',
 'cat_num_28',
 'cat_num_29',
 'cat_num_30',
 'cat_num_31',
 'cat_num_32']

In [13]:
user_app = pd.read_csv('user_app_actived.csv', header=None, names = ['uId', 'appId'])

In [14]:
user_app.head()

,uId,appId
0,1000008,a00105157#a00109386#a00134746#a0013952#a001451...
1,1000010,a00102782#a00107077#a00109386#a00134746#a00136...
2,1000011,a00158535#a00163116#a00170432#a00187480#a00224...
3,1000013,a00109386#a0011601#a0012768#a00136287#a0014516...
4,1000014,a00109386#a0011894#a0013059#a00134840#a0013578...


In [15]:
id2category_id_dict = dict()
for index, row in app_info.iterrows():
    if row['appId'] in id2category_id_dict:
        id2category_id_dict[row['appId']] += [row['category_id']]
    else:
        id2category_id_dict[row['appId']] = [row['category_id']]

In [16]:
1 in id2category_id_dict

False

In [17]:
user_id = user_app.uId.values

In [18]:
len(user_id)

2512500

In [19]:
sum_dict_index = ['sum_category_' + str(i) for i in np.arange(1, 33)]
min_dict_index =['min_category_' + str(i) for i in np.arange(1, 33)]
max_dict_index =['max_category_' + str(i) for i in np.arange(1, 33)]
mean_dict_index =['mean_category_' + str(i) for i in np.arange(1, 33)]
median_dict_index = ['median_category_' + str(i) for i in np.arange(1, 33)]
index_all =sum_dict_index +  min_dict_index + max_dict_index + mean_dict_index + median_dict_index
print(index_all)
for i in index_all:
    if 'min' in i or 'max' in i or 'sum' in i or 'median' in i:
        user_app[i] = 0
        user_app[i] = user_app[i].astype(int)
    else:
        user_app[i] = 0.0

['sum_category_1', 'sum_category_2', 'sum_category_3', 'sum_category_4', 'sum_category_5', 'sum_category_6', 'sum_category_7', 'sum_category_8', 'sum_category_9', 'sum_category_10', 'sum_category_11', 'sum_category_12', 'sum_category_13', 'sum_category_14', 'sum_category_15', 'sum_category_16', 'sum_category_17', 'sum_category_18', 'sum_category_19', 'sum_category_20', 'sum_category_21', 'sum_category_22', 'sum_category_23', 'sum_category_24', 'sum_category_25', 'sum_category_26', 'sum_category_27', 'sum_category_28', 'sum_category_29', 'sum_category_30', 'sum_category_31', 'sum_category_32', 'min_category_1', 'min_category_2', 'min_category_3', 'min_category_4', 'min_category_5', 'min_category_6', 'min_category_7', 'min_category_8', 'min_category_9', 'min_category_10', 'min_category_11', 'min_category_12', 'min_category_13', 'min_category_14', 'min_category_15', 'min_category_16', 'min_category_17', 'min_category_18', 'min_category_19', 'min_category_20', 'min_category_21', 'min_categ

In [20]:
id2category_id_dict

{'\\N': [19],
 'a00100007': [10, 5],
 'a00100109': [19],
 'a00100361': [16],
 'a00100402': [25],
 'a00100412': [3],
 'a00100436': [19],
 'a00100551': [16, 13],
 'a00100587': [24],
 'a00100609': [13],
 'a00100725': [13, 17],
 'a00100807': [17],
 'a00100823': [13, 17],
 'a00100924': [13, 17],
 'a00101036': [17, 13],
 'a0010117': [16],
 'a00101191': [29],
 'a00101205': [3, 13],
 'a00101266': [4],
 'a00101296': [13],
 'a00101298': [25],
 'a00101305': [13],
 'a00101316': [9],
 'a00101329': [9, 13],
 'a00101331': [9, 13, 25, 4],
 'a00101339': [29],
 'a00101347': [3, 13],
 'a00101378': [25],
 'a00101392': [25],
 'a00101522': [13],
 'a00101560': [6, 16],
 'a0010162': [13, 5],
 'a00101628': [13],
 'a00101632': [10],
 'a00101702': [17],
 'a00101750': [25, 3],
 'a00101827': [32],
 'a00101829': [32],
 'a00101831': [32],
 'a00101845': [32],
 'a00101992': [32, 5],
 'a00102154': [25],
 'a00102179': [24],
 'a00102193': [16],
 'a00102195': [16],
 'a00102196': [16],
 'a00102408': [13, 7],
 'a00102414': 

In [21]:
global count
count = 0
def apply_fre_fea(test):
    global count
    count += 1
    if count % 1000 == 0:
        print('count is', count)
    appid = test['appId'].split('#')
    num_dict = dict()
    for i in np.arange(1, 33):
        num_dict[i] = []
    for app_id in appid:
        for num in id2category_id_dict[app_id]:
            num_dict[num] += [eval('cat_num_' + str(num))[app_id]]
#             num_dict[num] += [app_info['category_id'+str(num)+'_level'][app_info.appId == app_id][app_info['category_id'+str(num)+'_level'] !=  0].values]
    num_dict_sum = []
    num_dict_min = []
    num_dict_max = []
    num_dict_mean = []
    num_dict_median = []
    for i in np.arange(1, 33):
        if len(num_dict[i]) == 0:
            num_dict_sum += [0]
            num_dict_min += [0]
            num_dict_max += [0]
            num_dict_mean += [0]
            num_dict_median += [0]
        else:
            num_dict_sum += [np.sum(num_dict[i])]
            num_dict_min += [np.min(num_dict[i])]
            num_dict_max += [np.max(num_dict[i])]
            num_dict_mean += [np.mean(num_dict[i])]
            num_dict_median += [np.median(num_dict[i])]
    test[sum_dict_index] = np.array(num_dict_sum)
    test[min_dict_index] = np.array(num_dict_min)
    test[max_dict_index] = np.array(num_dict_max)
    test[mean_dict_index] = np.array(num_dict_mean)
    test[median_dict_index] = np.array(num_dict_median)
    return test

In [22]:
user_app = user_app.apply(apply_fre_fea, axis=1)

count is 1000
count is 2000
count is 3000
count is 4000
count is 5000
count is 6000
count is 7000
count is 8000
count is 9000
count is 10000
count is 11000
count is 12000
count is 13000
count is 14000
count is 15000
count is 16000
count is 17000
count is 18000
count is 19000
count is 20000
count is 21000
count is 22000
count is 23000
count is 24000
count is 25000
count is 26000
count is 27000
count is 28000
count is 29000
count is 30000
count is 31000
count is 32000
count is 33000
count is 34000
count is 35000
count is 36000
count is 37000
count is 38000
count is 39000
count is 40000
count is 41000
count is 42000
count is 43000
count is 44000
count is 45000
count is 46000
count is 47000
count is 48000
count is 49000
count is 50000
count is 51000
count is 52000
count is 53000
count is 54000
count is 55000
count is 56000
count is 57000
count is 58000
count is 59000
count is 60000
count is 61000
count is 62000
count is 63000
count is 64000
count is 65000
count is 66000
count is 67000
coun

count is 520000
count is 521000
count is 522000
count is 523000
count is 524000
count is 525000
count is 526000
count is 527000
count is 528000
count is 529000
count is 530000
count is 531000
count is 532000
count is 533000
count is 534000
count is 535000
count is 536000
count is 537000
count is 538000
count is 539000
count is 540000
count is 541000
count is 542000
count is 543000
count is 544000
count is 545000
count is 546000
count is 547000
count is 548000
count is 549000
count is 550000
count is 551000
count is 552000
count is 553000
count is 554000
count is 555000
count is 556000
count is 557000
count is 558000
count is 559000
count is 560000
count is 561000
count is 562000
count is 563000
count is 564000
count is 565000
count is 566000
count is 567000
count is 568000
count is 569000
count is 570000
count is 571000
count is 572000
count is 573000
count is 574000
count is 575000
count is 576000
count is 577000
count is 578000
count is 579000
count is 580000
count is 581000
count is

count is 1031000
count is 1032000
count is 1033000
count is 1034000
count is 1035000
count is 1036000
count is 1037000
count is 1038000
count is 1039000
count is 1040000
count is 1041000
count is 1042000
count is 1043000
count is 1044000
count is 1045000
count is 1046000
count is 1047000
count is 1048000
count is 1049000
count is 1050000
count is 1051000
count is 1052000
count is 1053000
count is 1054000
count is 1055000
count is 1056000
count is 1057000
count is 1058000
count is 1059000
count is 1060000
count is 1061000
count is 1062000
count is 1063000
count is 1064000
count is 1065000
count is 1066000
count is 1067000
count is 1068000
count is 1069000
count is 1070000
count is 1071000
count is 1072000
count is 1073000
count is 1074000
count is 1075000
count is 1076000
count is 1077000
count is 1078000
count is 1079000
count is 1080000
count is 1081000
count is 1082000
count is 1083000
count is 1084000
count is 1085000
count is 1086000
count is 1087000
count is 1088000
count is 10890

count is 1513000
count is 1514000
count is 1515000
count is 1516000
count is 1517000
count is 1518000
count is 1519000
count is 1520000
count is 1521000
count is 1522000
count is 1523000
count is 1524000
count is 1525000
count is 1526000
count is 1527000
count is 1528000
count is 1529000
count is 1530000
count is 1531000
count is 1532000
count is 1533000
count is 1534000
count is 1535000
count is 1536000
count is 1537000
count is 1538000
count is 1539000
count is 1540000
count is 1541000
count is 1542000
count is 1543000
count is 1544000
count is 1545000
count is 1546000
count is 1547000
count is 1548000
count is 1549000
count is 1550000
count is 1551000
count is 1552000
count is 1553000
count is 1554000
count is 1555000
count is 1556000
count is 1557000
count is 1558000
count is 1559000
count is 1560000
count is 1561000
count is 1562000
count is 1563000
count is 1564000
count is 1565000
count is 1566000
count is 1567000
count is 1568000
count is 1569000
count is 1570000
count is 15710

count is 1995000
count is 1996000
count is 1997000
count is 1998000
count is 1999000
count is 2000000
count is 2001000
count is 2002000
count is 2003000
count is 2004000
count is 2005000
count is 2006000
count is 2007000
count is 2008000
count is 2009000
count is 2010000
count is 2011000
count is 2012000
count is 2013000
count is 2014000
count is 2015000
count is 2016000
count is 2017000
count is 2018000
count is 2019000
count is 2020000
count is 2021000
count is 2022000
count is 2023000
count is 2024000
count is 2025000
count is 2026000
count is 2027000
count is 2028000
count is 2029000
count is 2030000
count is 2031000
count is 2032000
count is 2033000
count is 2034000
count is 2035000
count is 2036000
count is 2037000
count is 2038000
count is 2039000
count is 2040000
count is 2041000
count is 2042000
count is 2043000
count is 2044000
count is 2045000
count is 2046000
count is 2047000
count is 2048000
count is 2049000
count is 2050000
count is 2051000
count is 2052000
count is 20530

count is 2477000
count is 2478000
count is 2479000
count is 2480000
count is 2481000
count is 2482000
count is 2483000
count is 2484000
count is 2485000
count is 2486000
count is 2487000
count is 2488000
count is 2489000
count is 2490000
count is 2491000
count is 2492000
count is 2493000
count is 2494000
count is 2495000
count is 2496000
count is 2497000
count is 2498000
count is 2499000
count is 2500000
count is 2501000
count is 2502000
count is 2503000
count is 2504000
count is 2505000
count is 2506000
count is 2507000
count is 2508000
count is 2509000
count is 2510000
count is 2511000
count is 2512000


In [23]:
user_app.head()

,uId,appId,sum_category_1,sum_category_2,sum_category_3,sum_category_4,sum_category_5,sum_category_6,sum_category_7,sum_category_8,...,median_category_23,median_category_24,median_category_25,median_category_26,median_category_27,median_category_28,median_category_29,median_category_30,median_category_31,median_category_32
0,1000008,a00105157#a00109386#a00134746#a0013952#a001451...,0,0,0,0,120,20,60,0,...,0.0,10.0,0.0,0.0,10.0,0.0,0.0,10.0,10.0,10.0
1,1000010,a00102782#a00107077#a00109386#a00134746#a00136...,0,0,0,0,142,0,108,0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,10.0
2,1000011,a00158535#a00163116#a00170432#a00187480#a00224...,0,0,10,0,40,0,0,0,...,0.0,10.0,10.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0
3,1000013,a00109386#a0011601#a0012768#a00136287#a0014516...,0,0,0,0,109,0,68,0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0,9.5,10.0
4,1000014,a00109386#a0011894#a0013059#a00134840#a0013578...,0,0,0,0,149,0,40,0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,10.0


In [24]:
user_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512500 entries, 0 to 2512499
Columns: 162 entries, uId to median_category_32
dtypes: float64(64), int64(97), object(1)
memory usage: 3.0+ GB


In [25]:
user_app.to_csv('./output/version2/app_actived_new_temp.csv', index=False)

In [26]:
user_app.drop(columns = 'appId', inplace=True)
user_app.head()

,uId,sum_category_1,sum_category_2,sum_category_3,sum_category_4,sum_category_5,sum_category_6,sum_category_7,sum_category_8,sum_category_9,...,median_category_23,median_category_24,median_category_25,median_category_26,median_category_27,median_category_28,median_category_29,median_category_30,median_category_31,median_category_32
0,1000008,0,0,0,0,120,20,60,0,0,...,0.0,10.0,0.0,0.0,10.0,0.0,0.0,10.0,10.0,10.0
1,1000010,0,0,0,0,142,0,108,0,0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,10.0
2,1000011,0,0,10,0,40,0,0,0,10,...,0.0,10.0,10.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0
3,1000013,0,0,0,0,109,0,68,0,0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0,9.5,10.0
4,1000014,0,0,0,0,149,0,40,0,0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,10.0


In [27]:
user_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512500 entries, 0 to 2512499
Columns: 161 entries, uId to median_category_32
dtypes: float64(64), int64(97)
memory usage: 3.0 GB


In [32]:
app_actived_index = pd.read_csv('./output/app_actived_new.csv', header=None, nrows=1).values
app_actived_index

array([['uId', 'category1', 'category2', 'category3', 'category4',
        'category5', 'category6', 'category7', 'category8', 'category9',
        'category10', 'category11', 'category12', 'category13',
        'category14', 'category15', 'category16', 'category17',
        'category18', 'category19', 'category20', 'category21',
        'category22', 'category23', 'category24', 'category25',
        'category26', 'category27', 'category28', 'category29',
        'category30', 'category31', 'category32', 'category_num1',
        'category_num2', 'category_num3', 'category_num4',
        'category_num5', 'category_num6', 'category_num8', 'fre',
        'app_num']], dtype=object)

In [38]:
app_actived_index[0]

array(['uId', 'category1', 'category2', 'category3', 'category4',
       'category5', 'category6', 'category7', 'category8', 'category9',
       'category10', 'category11', 'category12', 'category13',
       'category14', 'category15', 'category16', 'category17',
       'category18', 'category19', 'category20', 'category21',
       'category22', 'category23', 'category24', 'category25',
       'category26', 'category27', 'category28', 'category29',
       'category30', 'category31', 'category32', 'category_num1',
       'category_num2', 'category_num3', 'category_num4', 'category_num5',
       'category_num6', 'category_num8', 'fre', 'app_num'], dtype=object)

In [39]:
user_app[app_actived_index[0]] = pd.read_csv('./output/app_actived_new.csv')

In [40]:
user_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512500 entries, 0 to 2512499
Columns: 202 entries, uId to app_num
dtypes: float64(65), int64(137)
memory usage: 3.8 GB


In [41]:
user_app.to_csv('./output/version2/app_actived_new.csv', index=False)

# 第二次提取完成

# 第三次特征提取

In [2]:
app_info = pd.read_csv("app_info.csv", names=['appId','category'])
cats = list(set(app_info['category']))
category2id = dict(zip(sorted(cats), range(0,len(cats))))
app_info['category'] = app_info['category'].apply(lambda x: category2id[x])
app2id = dict(zip(app_info['appId'].values, app_info['category'].values))

In [3]:
def get_cat_have(x):
    x = set(x)
    cat = []
    for i in x:
        cat.append(app2id.setdefault(i, 40))
    return np.bincount(cat, minlength=41)
def get_cat_use(x):
    cat = []
    for i in x:
        cat.append(app2id.setdefault(i, 40))
    return np.bincount(cat, minlength=41)

In [255]:
%%time
now_df = pd.DataFrame()
now_df['uid'] = df['uid'].unique()
now_df = now_df.merge(df.groupby('uid')['appId'].count().to_frame(), how='left', on='uid')
now_df = now_df.merge(df.groupby('uid')['duration', 'times'].agg(['sum']), how='left', on='uid')
now_df = now_df.merge(df.groupby('uid')['appId', 'use_date'].agg(['nunique']), how='left', on='uid')
now_df = now_df.merge(df.groupby('uid')['duration','times'].agg(['mean','max','std']), how='left', on='uid') 
now_df.columns = ['uid','usage_appid_cnt','duration_cnt','times_cnt','usage_appid_cnt','usage_date_cnt','duration_mean','duration_max','duration_std','times_mean','times_max','times_std']
now_df['day_mean_time'] = now_df.apply(lambda x: x['duration_cnt']/x['usage_date_cnt'], axis=1)
now_df['day_mean_duration'] = now_df.apply(lambda x: x['times_cnt']/x['usage_date_cnt'], axis=1 )
for i in range(41):
    now_df['app_cat_have' + str(i)] = 0
    now_df['app_cat_use' + str(i)] = 0
now_df[['app_cat_have' + str(i) for i in range(41)]] = np.vstack(df.groupby('uid')['appId'].apply(lambda x: get_cat_have(x)).values)
now_df[['app_cat_use' + str(i) for i in range(41)]] = np.vstack(df.groupby('uid')['appId'].apply(lambda x: get_cat_use(x)).values)
now_df

Wall time: 577 ms


In [8]:
#处理数据量较大的user_app_usage.csv，结合app_info.csv简单统计得到appuseProcessed.csv作为特征
def processUserAppUsage():
    resTable = pd.DataFrame()
    reader = pd.read_csv("user_app_usage.csv", names=['uid','appId','duration','times','use_date'], iterator=True)
    last_df = pd.DataFrame()
    j = 1
    while True:
        try:
            print("index: {}".format(j))
            j+=1
            df = reader.get_chunk(1000000)
            df = pd.concat([last_df, df])
            idx = df.shape[0]-1
            last_user = df.iat[idx,0]
            while(df.iat[idx,0]==last_user):
                idx-=1
            last_df = df[idx+1:]
            df = df[:idx+1]

            now_df = pd.DataFrame()
            now_df['uid'] = df['uid'].unique()
            now_df = now_df.merge(df.groupby('uid')['appId'].count().to_frame(), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['duration', 'times'].agg(['sum']), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['appId', 'use_date'].agg(['nunique']), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['duration','times'].agg(['mean','max','std']), how='left', on='uid') 
            now_df.columns = ['uid','usage_appid_cnt','duration_cnt','times_cnt','usage_appid_cnt','usage_date_cnt','duration_mean','duration_max','duration_std','times_mean','times_max','times_std']
            now_df['day_mean_time'] = now_df.apply(lambda x: x['duration_cnt']/x['usage_date_cnt'], axis=1)
            now_df['day_mean_duration'] = now_df.apply(lambda x: x['times_cnt']/x['usage_date_cnt'], axis=1 )
            for i in range(41):
                now_df['app_cat_have' + str(i)] = 0
                now_df['app_cat_use' + str(i)] = 0
            now_df[['app_cat_have' + str(i) for i in range(41)]] = np.vstack(df.groupby('uid')['appId'].apply(lambda x: get_cat_have(x)).values)
            now_df[['app_cat_use' + str(i) for i in range(41)]] = np.vstack(df.groupby('uid')['appId'].apply(lambda x: get_cat_use(x)).values)
            resTable = pd.concat([resTable, now_df])
        except StopIteration:
            break
    
    resTable.to_csv("appuseProcessed.csv",index=0)
    
    print("Iterator is stopped")

In [9]:
processUserAppUsage()

index: 1
index: 2
index: 3
index: 4
index: 5
index: 6
index: 7
index: 8
index: 9
index: 10
index: 11
index: 12
index: 13
index: 14
index: 15
index: 16
index: 17
index: 18
index: 19
index: 20
index: 21
index: 22
index: 23
index: 24
index: 25
index: 26
index: 27
index: 28
index: 29
index: 30
index: 31
index: 32
index: 33
index: 34
index: 35
index: 36
index: 37
index: 38
index: 39
index: 40
index: 41
index: 42
index: 43
index: 44
index: 45
index: 46
index: 47
index: 48
index: 49
index: 50
index: 51
index: 52
index: 53
index: 54
index: 55
index: 56
index: 57
index: 58
index: 59
index: 60
index: 61
index: 62
index: 63
index: 64
index: 65
index: 66
index: 67
index: 68
index: 69
index: 70
index: 71
index: 72
index: 73
index: 74
index: 75
index: 76
index: 77
index: 78
index: 79
index: 80
index: 81
index: 82
index: 83
index: 84
index: 85
index: 86
index: 87
index: 88
index: 89
index: 90
index: 91
index: 92
index: 93
index: 94
index: 95
index: 96
index: 97
index: 98
index: 99
index: 100
index: 1

In [10]:
appuseProcessed = pd.read_csv("appuseProcessed.csv")
appuseProcessed_temp = pd.read_csv("output/appuseProcessed.csv")

In [14]:
appuseProcessed.head()

,uid,usage_appid_cnt,duration_cnt,times_cnt,usage_appid_cnt.1,usage_date_cnt,duration_mean,duration_max,duration_std,times_mean,...,app_cat_have37,app_cat_use37,app_cat_have38,app_cat_use38,app_cat_have39,app_cat_use39,app_cat_have40,app_cat_use40,category,usage_most_used_category
0,1000364,198,390756.0,6390.0,13,30,1973.515152,44845.0,4600.297805,32.272727,...,2,22,0,0,1,14,0,0,8,18.0
1,1000387,491,975205.0,30938.0,41,30,1986.160896,24954.0,3338.793126,63.010183,...,3,45,1,19,3,20,1,24,14,18.0
2,1000872,406,349616.0,22877.0,29,30,861.123153,11875.0,1912.555912,56.347291,...,3,47,1,12,5,59,1,4,12,18.0
3,1000951,223,374179.0,6360.0,21,30,1677.932735,17073.0,2826.470570,28.520179,...,2,26,0,0,1,9,1,4,11,18.0
4,1001091,1019,887591.0,35906.0,160,30,871.041217,33739.0,2445.063938,35.236506,...,13,123,1,4,18,104,11,52,20,18.0


In [12]:
appuseProcessed_temp.columns

Index(['uid', 'usage_cnt', 'usage_appid_cnt', 'usage_date_cnt',
       'duration_mean', 'duration_max', 'duration_std', 'times_mean',
       'times_max', 'times_std', 'category', 'usage_most_used_category'],
      dtype='object')

In [13]:
appuseProcessed['category'] = appuseProcessed_temp['category']
appuseProcessed['usage_most_used_category'] = appuseProcessed_temp['usage_most_used_category']

In [15]:
appuseProcessed.to_csv("appuseProcessed.csv",index=0)

# 第五次特征抽取

In [2]:
train_data = pd.read_csv('./output/version4/train_data.csv')

In [3]:
train_data.drop(columns=['uid','gender','ramCapacity','ramLeftRation','romCapacity','romLeftRation','fontSize','os','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes','FFuncSum','city_num','prodName_num','color_num','carrier_num','ct_num','usage_cnt','usage_appid_cnt','usage_date_cnt','duration_mean','duration_max','duration_std','times_mean','times_max','times_std','category','usage_most_used_category','age_group' ], inplace=True)

In [4]:
train_data2 = pd.read_csv('./output/version3/train_data.csv')

In [5]:
train_data2 = pd.concat([train_data2, train_data], axis=1)

In [6]:
train_data2.head()

,uid,gender,ramCapacity,ramLeftRation,romCapacity,romLeftRation,fontSize,os,bootTimes,AFuncTimes,...,1,2,3,4,5,6,7,8,9,tfidf_sum
0,1000001,0,3.0,0.43,32.0,0.46,1.15,8.0,108,0.0,...,0.004546,0.004547,0.004546,0.253997,0.004546,0.004546,0.709631,0.004547,0.004547,4.143144
1,1000011,0,0.0,0.00,0.0,0.00,0.00,8.1,0,0.0,...,0.005886,0.005885,0.005883,0.005884,0.005884,0.947042,0.005884,0.005886,0.005883,3.218423
2,1000015,1,3.0,0.34,32.0,0.06,1.30,8.0,12,0.0,...,0.005001,0.005002,0.005001,0.754737,0.205253,0.005001,0.005002,0.005001,0.005002,3.189480
3,1000019,0,2.0,0.00,17.0,0.00,0.00,8.1,0,0.0,...,0.033333,0.033341,0.033334,0.033366,0.033334,0.033343,0.033335,0.033340,0.699939,1.256031
4,1000023,1,2.0,0.34,16.0,0.06,1.00,7.0,5,0.0,...,0.007143,0.007144,0.007143,0.935702,0.007144,0.007144,0.007147,0.007145,0.007144,2.970135


In [7]:
temp = train_data2.age_group
train_data2.drop(columns=['age_group'], inplace=True)
train_data2['age_group'] = temp

In [13]:
train_data2.to_csv('./output/version5/train_data.csv')

In [14]:
test_data = pd.read_csv('./output/version4/test_data.csv')
test_data.drop(columns=['uid','gender','ramCapacity','ramLeftRation','romCapacity','romLeftRation','fontSize','os','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes','FFuncSum','city_num','prodName_num','color_num','carrier_num','ct_num','usage_cnt','usage_appid_cnt','usage_date_cnt','duration_mean','duration_max','duration_std','times_mean','times_max','times_std','category','usage_most_used_category'], inplace=True)
test_data2 = pd.read_csv('./output/version3/test_data.csv')
test_data2 = pd.concat([test_data2, test_data], axis=1)
test_data2.head()

,uid,gender,ramCapacity,ramLeftRation,romCapacity,romLeftRation,fontSize,os,bootTimes,AFuncTimes,...,1,2,3,4,5,6,7,8,9,tfidf_sum
0,1000002,1,0.0,0.26,0.0,0.59,1.0,9.0,14,0.0,...,0.001429,0.001429,0.549596,0.001429,0.001429,0.001429,0.001429,0.100566,0.001429,7.322537
1,1000003,0,8.0,0.35,128.0,0.60,1.0,9.0,13,1.0,...,0.141031,0.001235,0.082345,0.140067,0.001235,0.001235,0.629149,0.001235,0.001235,7.855082
2,1000004,1,3.0,0.36,32.0,0.09,1.0,8.0,57,0.0,...,0.003704,0.648274,0.003704,0.003705,0.003705,0.003704,0.003705,0.003705,0.003704,4.171360
3,1000005,0,6.0,0.00,68.0,0.00,0.0,9.0,0,0.0,...,0.632149,0.344987,0.002858,0.002859,0.002858,0.002858,0.002858,0.002858,0.002858,5.266798
4,1000006,0,2.0,0.19,16.0,0.42,0.0,6.0,33,0.0,...,0.009092,0.009093,0.009092,0.621194,0.306066,0.009093,0.009093,0.009092,0.009092,2.614293


In [15]:
test_data2.to_csv('./output/version5/test_data.csv')